# Example prompts for `ncoder`

Load the extension

In [ ]:
%load_ext ncoder

## List of example prompts

- Write a Python program that connects to a webpage and extract it's text
- Write a x86 assembly program that sorts a list
- Write a TxtAI RAG example
- Update to the qwen3 4B model for RAG
- Build a txtai program that reads the txtai-arxiv embeddings database and uses the results for a RAG pipeline
- Use TxtAI to parse the TxtAI github and build an embeddings index
- Write a TxtAI program that translates English to French
- Write a txtai program that reads a website and summarizes it
- Write a Python program that builds a networkx graph. Make it a class and add a main method to show how to use it.
- Write a TxtAI program converts PDFs to text chunks
- Show me a basic example on how to build a binary tree in Python
- Research txtai embeddings and the simplest program using it
- Write a Python program that acts like the `ls` command
- Write an opencode python sdk example
- Write a TxtAI program that creates captions for images

In [ ]:
%%ncoder Write a Python Hello World

# Hello World program in Python
print("Hello, World!")

In [ ]:
%%ncoder Write a simple binary tree program in Python

class Node:
    # Node class for binary tree
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

class BinaryTree:
    # Binary tree implementation
    def __init__(self):
        self.root = None
    
    def insert(self, data):
        # Insert a new node
        if self.root is None:
            self.root = Node(data)
        else:
            self._insert_recursive(self.root, data)
    
    def _insert_recursive(self, node, data):
        # Helper method for recursive insertion
        if data < node.data:
            if node.left is None:
                node.left = Node(data)
            else:
                self._insert_recursive(node.left, data)
        else:
            if node.right is None:
                node.right = Node(data)
            else:
                self._insert_recursive(node.right, data)
    
    def inorder_traversal(self):
        # Inorder traversal (left-root-right)
        result = []
        self._inorder_recursive(self.root, result)
        return result
    
    def _inorder_recursive(self, node, result):
        # Helper for inorder traversal
        if node:
            self._inorder_recursive(node.left, result)
            result.append(node.data)
            self._inorder_recursive(node.right, result)
    
    def preorder_traversal(self):
        # Preorder traversal (root-left-right)
        result = []
        self._preorder_recursive(self.root, result)
        return result
    
    def _preorder_recursive(self, node, result):
        # Helper for preorder traversal
        if node:
            result.append(node.data)
            self._preorder_recursive(node.left, result)
            self._preorder_recursive(node.right, result)
    
    def postorder_traversal(self):
        # Postorder traversal (left-right-root)
        result = []
        self._postorder_recursive(self.root, result)
        return result
    
    def _postorder_recursive(self, node, result):
        # Helper for postorder traversal
        if node:
            self._postorder_recursive(node.left, result)
            self._postorder_recursive(node.right, result)
            result.append(node.data)

# Example usage
if __name__ == "__main__":
    tree = BinaryTree()
    
    # Insert nodes
    tree.insert(50)
    tree.insert(30)
    tree.insert(70)
    tree.insert(20)
    tree.insert(40)
    tree.insert(60)
    tree.insert(80)
    
    # Print traversals
    print("Inorder traversal:", tree.inorder_traversal())
    print("Preorder traversal:", tree.preorder_traversal())
    print("Postorder traversal:", tree.postorder_traversal())

In [ ]:
%%ncoder Write a TxtAI RAG Example

from txtai.embeddings import Embeddings
from txtai.pipeline import RAG

# Create embeddings model
embeddings = Embeddings({
    "path": "sentence-transformers/all-MiniLM-L6-v2",
    "content": True
})

# Sample documents for knowledge base
documents = [
    "Python is a high-level programming language known for its simplicity.",
    "Machine learning is a subset of artificial intelligence that enables computers to learn.",
    "Natural language processing helps computers understand human language.",
    "Deep learning uses neural networks with multiple layers to process data.",
    "Data science combines statistics, programming, and domain expertise.",
    "TensorFlow is an open-source machine learning framework developed by Google.",
    "PyTorch is a deep learning framework developed by Facebook's AI Research lab.",
    "Computer vision enables machines to interpret and understand visual information.",
    "Reinforcement learning trains agents to make decisions through rewards and punishments.",
    "Feature engineering is the process of creating new variables from existing data."
]

# Index the documents
embeddings.index([(i, doc, None) for i, doc in enumerate(documents)])

# Create RAG pipeline
rag = RAG(
    embeddings,
    "Qwen/Qwen3-0.6B",
    template="""Answer the following question using only the provided context. 
    If the context doesn't contain the answer, say "I don't have enough information to answer."
    
    Context: {context}
    
    Question: {question}
    
    Answer:"""
)

# Example queries
if __name__ == "__main__":
    questions = [
        "What is machine learning?",
        "Which framework was developed by Google?",
        "How does reinforcement learning work?",
        "What programming language is mentioned in the documents?"
    ]
    
    for question in questions:
        print(f"\nQuestion: {question}")
        answer = rag(question)
        print(f"Answer: {answer}")
        print("-" * 50)

In [ ]:
%%ncoder Write a txtai program that reads a website and summarizes it

from txtai.pipeline import Summary
import requests
from bs4 import BeautifulSoup
import re

def extract_webpage_content(url):
    # Extract main text content from a webpage
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        
        # Parse HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Remove script and style elements
        for script in soup(["script", "style", "nav", "footer", "header", "aside"]):
            script.decompose()
        
        # Extract text from main content areas
        content_selectors = [
            'article', 'main', '.content', '.post-content', 
            '.entry-content', '#content', '.article-content'
        ]
        
        text_content = ""
        
        # Try to find main content area first
        for selector in content_selectors:
            main_content = soup.select_one(selector)
            if main_content:
                text_content = main_content.get_text(separator=' ', strip=True)
                break
        
        # Fallback to body if no specific content found
        if not text_content:
            text_content = soup.get_text(separator=' ', strip=True)
        
        # Clean up text
        text_content = re.sub(r'\s+', ' ', text_content)  # Remove extra whitespace
        text_content = re.sub(r'[\n\r\t]+', ' ', text_content)  # Remove newlines and tabs
        
        return text_content.strip()
        
    except Exception as e:
        raise Exception(f"Error fetching webpage: {str(e)}")

def summarize_website(url):
    # Create summary pipeline
    summary_pipeline = Summary()
    
    # Extract content from website
    print(f"Extracting content from: {url}")
    content = extract_webpage_content(url)
    
    # Check if we got enough content
    if len(content) < 100:
        raise Exception("Not enough content to summarize")
    
    print(f"Extracted {len(content)} characters of content")
    
    # Generate summary
    print("Generating summary...")
    summary = summary_pipeline(content)
    
    return summary

def main():
    # Example websites to summarize
    websites = [
        "https://github.com/neuml/txtai"
    ]
    
    for url in websites:
        try:
            print(f"\n{'='*60}")
            print(f"Summarizing: {url}")
            print(f"{'='*60}")
            
            summary = summarize_website(url)
            
            print(f"\nSummary:")
            print(f"{summary}")
            print(f"\n{'='*60}\n")
            
        except Exception as e:
            print(f"Error summarizing {url}: {str(e)}")
            continue

if __name__ == "__main__":
    # Demo mode with predefined websites
    main()